# Progress Report: [Blog-Sentiment Analysis](https://github.com/Data-Science-for-Linguists-2019/Blog-Sentiment-Analysis)

To learn more about the data and see my previous analysis, refer to [Progress Report #2](https://github.com/Data-Science-for-Linguists-2019/Blog-Sentiment-Analysis/blob/master/progress_report_part2.ipynb).

# Loading the data

In [1]:
# Import required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
%pprint

Pretty printing has been turned OFF


In [2]:
dir = '/users/eva/Documents/Data_Science/Blog-Sentiment-Analysis/'

In [3]:
blogdata = pd.read_csv(dir + 'data/blogtext.csv')

In [4]:
newcolumns = ['id', 'gender', 'age', 'industry', 'sign', 'date', 'text']
blogdata.columns = newcolumns
blogdata.head(3)

,id,gender,age,industry,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...


# Data analysis
### Goals of my analysis
There are three things I would like to investigate in this data. 
1. Word frequencies
2. Blog topics
3. Blog sentiment

I already looked at [word frequencies](https://github.com/Data-Science-for-Linguists-2019/Blog-Sentiment-Analysis/blob/master/progress_report_part2.ipynb) in Progress Report #2, though I'm not super happy with my results. I experimented with [topic clustering using LDA (Latent Dirichlet Allocation) and scikit-learn](https://github.com/Data-Science-for-Linguists-2019/Blog-Sentiment-Analysis/blob/master/progress_report_part2.ipynb). 

For Progress Report #3, I am going to begin by exploring blog sentiment. If I have time, I will revisit the previous two goals of my analysis.

## Sentiment analysis

Ok so, it appears the vast majority of sentiment analysis groups texts into "positive" and "negative". I would really like to go deeper than that if possible, but I'll try positive and negative classification first.

Since I am really not interested in labeling even a portion of the 681,284 blogs in this dataset for sentiment, I think I am going to try using some already trained models or wordlists with sentiment mapping. Many sentiment models were trained on traditional corpus data, using text from news and books. These models don't incorporate things like emoticons :-) and other internet-specific language which would be found in the blog dataset. In order to get the best sentiment judgments, I specifically looked for models trained on more modern corpora.

### First try: VADER
VADER (Valence Aware Dictionary and sEntiment Reasoner) is "a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media". It is also part of NLTK, which is convenient. It looks like it can be used on a whole text without tokenization.
Credit:
> Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.

In [31]:
print(blogdata.text[30000])
text = blogdata.text[30000]

       well another days gone past and a major highlight of today has been two mass nose bleeds, while the blood is an itresting liquid its rather annoting when its streaning from your nose instead of someones neck. also today was the first full veiwing of the transit of venus across the sun whitnessed by mankind so though it may have been a pathetic black dot moving across the sun it was a once in a life time event and I'm slighty happyer that i've seen what some people can never see.  well with two nosebleeds, a mathes exam, a good roasting at the hands of the sun, an inability to focus properly and a ponding headache coming on I'm not feeling particly good tonight. but i've found my way back into hack this site which is a cool website which basicly gives the basics to the advanced on web hacking if your intrested its  urlLink hack-this-site dot org  aI'm quite happy with myself as im now twice as far as i got last time and i read loads of help articals then  well i dont feel like ty

After doing some poking around I am going to try this blog - it seems pretty negative, but it also uses a lot of sarcasm and understatement, as well as mispellings, which might be an issue. And though it does focus on the negative it also mentions some happier things. Let's see how VADER classifies this.

In [30]:
nltk.sentiment.util.demo_vader_instance(text)

{'neg': 0.052, 'neu': 0.798, 'pos': 0.149, 'compound': 0.9709}


VADER ranks sentiment using a score between -1 and 1. Greater than 0.5 is positive, less than -0.5 is negative, and anything in between is neutral. `nltk.sentiment.util.demo_vader_instance` returns the percentage of the text considered negative, percentage considered neutral, percentage considered positive, and lastly the overall score for the document (`compound`). VADER did a not supergreat job, since it returned a very very positive score for a negative-seeming document, as well as categorizing most of the statements as "neutral". I considered this to be a pretty emotionally charged blog. Let's keep exploring VADER anyway and see how it does.

In [35]:
blogdata.text[412412]

"           I am pondering the idea of attempting to learn XML again, to make the site a bit more consistiant, in terms of tags, text sizes, and styles... Fear-not, it wont cut into the time I spen Dev'ing, just improve the work I do :)         "

In [36]:
nltk.sentiment.util.demo_vader_instance(blogdata.text[412412])

{'neg': 0.0, 'neu': 0.827, 'pos': 0.173, 'compound': 0.7727}


This statement seems pretty darn neutral to me, though there is a smiley face. The classifier seems to lean positive.

In [37]:
# let's try a longer text
blogdata.text[300000]

'       Wat my feeling toward Sheffield??? Cold, cool and a bit of disappointment.   Cold – although now is summer, but the weather was quite cold especially the 1st day. It was a gloomy day and rain late of the day. The temperature was below 10 degree Celsius. Then the other days was like the weather in Genting Highlands and sometime even cooler then Genting… but I quite like the weather here, u can walk for hours with out any sweat in the afternoon. Believe it or not….  Cool – the cars, building, scenery and bla blab la were cool. After a few days here, I can see all my dream cars, Ferrari, Lambogini, Lotus, GTO, FTO, Impreza, BMW (the model tat drove by James Bond) and many many more tat I dunno the name. however, there is proton here also. Oh ya, kancil also got. Taken photo somemore… hahaha : p  oh.. bout the building, most of the building may seem a bit old, but inside the building was very nice and class leh…. E.g. Sheffield Hallam Uni… : )  Disappointment – cant get into the ap

In [38]:
nltk.sentiment.util.demo_vader_instance(blogdata.text[300000])

{'neg': 0.057, 'neu': 0.84, 'pos': 0.103, 'compound': 0.9921}


It seems to work pretty good on longer texts. I guess this text is positive overall? It's hard to tell. 

### Running VADER on 10k blog sample

I want to explore some ways to extract information from sentiment analysis. I'm going to do this on a smaller sample of the data before trying them out on the whole dataset.

In [59]:
sample = blogdata.sample(10000)

In [60]:
sample['lines'] = sample.text.map(nltk.sent_tokenize)

In [61]:
sample.head(3)

,id,gender,age,industry,sign,date,text,lines
608072,572994,male,24,Religion,Libra,"14,June,2004",whoever serves me must follo...,[ whoever serves me must foll...
63959,3931176,female,14,indUnk,Sagittarius,"25,July,2004",I hate my dad. God. Why is he so obsess...,"[ I hate my dad., God., Why is he so obs..."
10690,873288,female,23,Arts,Capricorn,"04,mars,2004",I just tried to post a monster blog...,[ I just tried to post a monster blo...


In [62]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [63]:
sia = SentimentIntensityAnalyzer()

In [79]:
score = sia.polarity_scores(sample['text'][608072])

In [80]:
score['compound']

0.9805

In [87]:
def summarize_sentiment(df):
    all_scores = []
    for x in df.index:
        score = sia.polarity_scores(df['text'][x])
        if score['compound'] >= 0.5:
            all_scores.append('positive')
        if score['compound'] <= -0.5:
            all_scores.append('negative')
        else:
            all_scores.append('neutral')
    return all_scores 

In [91]:
len(sent_scores)

15436

In [90]:
sample['sentiment'] = summarize_sentiment(sample)

ValueError: Length of values does not match length of index

In [89]:
sample.head()

['positive', 'neutral', 'negative', 'neutral', 'neutral', 'positive', 'neutral', 'positive', 'neutral', 'negative']

### First try: AFINN
[AFINN](https://github.com/fnielsen/afinn) is described as a "Wordlist-based approach for sentiment analysis". The wordlist was created using data from Twitter. It contains basically every emoticon and things like "wowow" (4/5 on the positive scale) and "wtfff" (-4/-5 on the negative scale). I read the paper accompanying the wordlist and thought it sounded like a potential fit for the blog data.

> Finn Årup Nielsen, "A new ANEW: evaluation of a word list for sentiment analysis in microblogs", Proceedings of the ESWC2011 Workshop on 'Making Sense of Microposts': Big things come in small packages. Volume 718 in CEUR Workshop Proceedings: 93-98. 2011 May. Matthew Rowe, Milan Stankovic, Aba-Sah Dadzie, Mariann Hardey (editors)